# Yolov3 Dataloader

Author: Kevin Yu (github.com/yqlbu) \
Email: kevinyu211@yahoo.com \
 \
** This tool is tailored for those who want to train their custom dataset on a Yolov3 Model. \

** Make sure you read the instructions for each step carefully !!!

** If you following the instructions below step by step, it will generate a new trained-weight in the end, and you may download it to your local machine. \

** In the github repo, you can find a "demo.sh" to test the result. \
 \



#### Good Luck!

 # Setup

### Install cuDNN-10.0 and other dependecies

In [ ]:
# Download the cuDNN driver
!wget https://hidden-boat-623a.keviny-cloud.workers.dev/Software/cudnn-10.0-linux-x64-v7.5.0.56.tgz -O cudnn-10.0-linux-x64-v7.5.0.56.tgz
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# install python-nvcc plugin
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

# check if installed successfully
!/usr/local/cuda/bin/nvcc --version

### Clone Yolov3-Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/yolov3.weights -q --show-progress --no-clobber -O /content/darknet/yolov3.weights
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/yolov3-tiny.weights -q --show-progress --no-clobber -O /content/darknet/yolov3-tiny.weights
!wget https://objectstorage.ca-toronto-1.oraclecloud.com/n/yzpqsgba6ssd/b/bucket-20200415-0121/o/darknet53.conv.74 -q --show-progress --no-clobber -O /content/darknet/cfg/darknet53.conv.74

### Compile YOLOv3-Darknet with CUDA 10.0

In [ ]:
!export PATH=/usr/local/cuda-10.0/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
%cd /content/darknet
!sed -i 's|GPU=0|GPU=1|' Makefile
!sed -i 's|CUDNN=0|CUDNN=1|' Makefile
!sed -i 's|OPENCV=0|OPENCV=1|' Makefile

In [ ]:
!make

### Clone the project from Github

In [ ]:
%cd /content
!git clone https://github.com/yqlbu/yolov3-dataloader
!mkdir /content/yolov3-dataloader/backup

# Upload dataset

*** Please follow the steps below to upload your dataset to the data folder located at  '/content/data'

Step #0: Prepare your own dataset

- On your local machine, create a new folder named "data"
- Put all the images and the ".xml" files associated with each image into this data folder
- Zip the data folder

*** Noted: 
- You should have a zip file called "data.zip" that contains all the images (.png/.jpg/.jpeg) aligned with the .xml files to be trained before moving to the next step.

Step #1: Mount your Google Drive

To do so, you may simply click the Mount Drive button on the left column

Step #2: Unzip the data file from your Google Drive to the 'yolov3-dataloader' directory
- Note: if your path contains a " " (space), you should add a '\ ' between characters
- eg: '/content/drive/Shared\ drives/dataset/data.zip'
- To easily find the path just simply find the data.zip file in your drive and right-click the "copy path"
- Please make sure you correctly find your file path before executing this cell. Otherwise, it won't work.

In [ ]:
!unzip <DATA_PATH> -d /content/yolov3-dataloader
!echo '  Finished loading dataset !'

*** Noted: If the above commands excuted correctly, you may click the 'refresh button' on the left column, and you will see the data folder created inside the 'yolov3-dataloader' directory

*** Now you should be good to move forward ! Good Luck !

# Create config files for training

*** Notes (Please read carefully before you excute the cells below):

 - Please make sure you set config parameters below corrently. Otherwise, you will not be able to activate the training process.
 - MODEL_NAME is the name of your new model
 - CLASS_NAME is associated with all the classes that your dataset contains, please fill in the CLASS_NAME list correctly with your classes
 - CLASS_NUM is associated with the total number of classes that your dataset contains
 - MAX_BATCHES is the total number of iterations in the training
 - If you set all the parameters mentioned above properly, you should be good to move forward

In [ ]:
import os

#New Model Parameters

#Replace NEW_MODEL_NAME with your new model name
MODEL_NAME = 'mask'
#Put your classes tag below
CLASS_NAME = [
    "Mask",
    "No_Mask"
]
#Replace # with your classes number (int), 2 is set as the default value
CLASS_NUM = 2
#Replace your desired batch # (6000 as default)
#Ideally the MAX_BATCHES = 2500*CLASS_NUM
MAX_BATCHES = 5000

%cd /content/yolov3-dataloader

In [ ]:
#class.name
with open(MODEL_NAME+'.names', "a+") as file:
    file.seek(0)
    for i in CLASS_NAME:
        file.write(i + '\n')
    print(MODEL_NAME+'.names'+' has been successfully created!')
    file.close()

In [ ]:
#class.data
with open(MODEL_NAME+'.data', "a+") as file:
    file.seek(0)
    file.write('classes = ' + str(CLASS_NUM) + '\n')
    file.write('train = /content/yolov3-dataloader/train.txt' + '\n')
    file.write('valid = /content/yolov3-dataloader/test.txt' + '\n')
    file.write('backup = /content/yolov3-dataloader/backup' + '\n')
    file.write('names = /content/yolov3-dataloader/' + MODEL_NAME + '.names' + '\n')
    print(MODEL_NAME+'.data'+' has been successfully created!')
    file.close()

In [ ]:
#dataDispatch.py
with open('dataDispatch.py', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('dataDispatch.py', "w") as file:
    all_lines[7] = 'classes = ' + str(CLASS_NAME) + '\n'
    print(all_lines[7].strip())
    file.writelines(all_lines)
    print('dataDispatch.py'+' has been successfully modified!')
    file.close()

In [ ]:
#class-yolov3-tiny.cfg
with open('temp-config.txt', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open(MODEL_NAME+'-yolov3-tiny.cfg', "w") as file:
    #modify class #
    all_lines[134] = 'classes=' + str(CLASS_NUM) + '\n'
    all_lines[176] = 'classes=' + str(CLASS_NUM) + '\n'
    #modify max-batches
    all_lines[19] = 'max_batches = ' + str(MAX_BATCHES) + '\n'    
    #modify filter ## filter = (CLASS_NUM+5) *3 ##
    all_lines[126] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    all_lines[170] = 'filters=' + str((CLASS_NUM+5) *3) + '\n'
    #overwrite all
    file.writelines(all_lines)
    print(MODEL_NAME+'-yolov3-tiny.cfg'+' has been successfully created!')
    file.close()

In [ ]:
#train.sh
with open('train.sh', "r") as file:
    file.seek(0)
    all_lines = file.readlines()
    file.close()
with open('train.sh', "w") as file:
    all_lines[5] = '$PRJ_PATH/' + MODEL_NAME + '.data \\' + '\n'
    all_lines[6] = '$PRJ_PATH/' + MODEL_NAME+'-yolov3-tiny.cfg' + ' \\' + '\n'
    file.writelines(all_lines)
    print('train.sh'+' has been successfully modified!')
    file.close()

# Training

*** Notes:

 - Hit the "Refresh" button on the left column, and then navigate to the "/yolov3-dataloader" directory
 - Check if the directory contains the ".data", the ".names", and the ".cfg" files. If you miss one or more of the files, please check the instructions from the above steps.
 - Now you should be good to activate the training process, good luck !

In [ ]:
#start training
!export PATH=/usr/local/cuda-10.0/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
%cd /content/yolov3-dataloader/
!python dataDispatch.py
!source train.sh
!echo '*** Congratulations! Training has completed. Enjoy! ***'

*** Noted: \
- Once the training has finished, you may now go to the '/content/yolov3-dataload/backup' directory. \
- Download the final weights to your local machine. Enjoy !

# Demo

** Now that you have successfully trained your own dataset with the YOLOv3 model, it is time to test it out ! \
 \
** Please follow the steps below:

 - Download the "train.txt", "test.txt", ".data", ".names", and ".cfg" from "/content/yolov3-dataloader", and the "final_weight" from the "/content/yolov3-dataloader/backup" directory (in total 6 files need to be downloaded) \
 - On your local machine, navigate to the "/yolov3-dataloader" directory, 
  - Open the .data file downloaded from Colab. 
  - Set the absolute path associated with the "train.txt", "test.txt", ".weight", and ".names". 
  - Save & Close \
 - On your local machine, navigate to the "/yolov3-dataloader" directory, then type the following commands \

   [a] nano demo.sh \
     --> You need to set the name of your .data, .cfg, and weight files in the demo.sh. \
     --> Also, you need to set the absolute path of both the darknet and the yolov3-dataloader. Save & Exit. \
   [b] chmod +x demo.sh \
   [c] ./demo.sh

**  To import stream data from a media file, you need to place the media file (.mp4, .mkv, .avi) inside the directory, and then replace "-c 0 -thresh .5" with "-ext-out $PRJ_PATH\YOUR_MEDIA" to the last line of "demo.sh" \
 \
** Now, you should be good to go ! Enjoy XD! \
** If you have any suggestions or comments, please contact me via kevinyu211@yahoo.com


